In [ ]:
import sys
import numpy as np
from math import ceil
import tensorflow as tf
from tensorflow.math import log
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB2
from tensorflow import zeros_like,reduce_sum,divide,exp,expand_dims
from tensorflow.keras.layers import Input, TimeDistributed,Flatten,Dense,Dropout,Reshape


sys.path.append('/kaggle/input/mlware-preprocessed')
from DataHelper import *

In [ ]:
helpMe()

In [ ]:
x_train,x_test = load_images("/kaggle/input/mlware-preprocessed/Landmarks_train",valid_split=0.2)

In [ ]:

y=PermutationMatrix("/kaggle/input/mlware-preprocessed/Preprocessed_landmarks.csv")
y_train,y_test=y[:ceil(y.shape[0]*0.8)],y[ceil(y.shape[0]*0.8):]


In [ ]:
newx_train,newy_train=AugmentedData(x_train,y_train,6000,augments_per_image=15)

In [ ]:
newx_test,newy_test=AugmentedData(x_test,y_test,400,augments_per_image=15)

In [ ]:
class SinkhornLayer(tf.keras.layers.Layer):
    def __init__(self, epsilon=0.1, n_iters=10):
        super().__init__()
        self.epsilon = epsilon
        self.n_iters = n_iters

    def sinkhorn(self, x):
        # Sinkhorn iterations
        self.u = zeros_like(x[:, 0])
        self.v = zeros_like(x[0])
        for i in range(self.n_iters):
            self.u = -self.epsilon * (log(reduce_sum(exp(divide(x - expand_dims(self.v, axis=1), self.epsilon)), axis=2))) + self.u
            self.v = -self.epsilon * (log(reduce_sum(exp(divide(x - expand_dims(self.u, axis=2), self.epsilon)), axis=1))) + self.v

        # Compute the sinkhorn matrix
        sinkhorn_matrix = exp(divide(x - expand_dims(self.u, axis=2) - expand_dims(self.v, axis=1), self.epsilon))

        return sinkhorn_matrix
def create_siamese_network():
    # Define the input layer
    inputs = Input(shape=(36, 50, 50, 3))
    
    # Define the shared layers of the Siamese network using EffiecientNet architecture
    shared_layers = EfficientNetB2(
        input_shape=(50, 50, 3),
        include_top=False,
        weights="imagenet"
    )
    shared_layers.trainable = False

    # Apply the shared layers to each input image
    x = TimeDistributed(shared_layers)(inputs)
    x = Flatten()(x)

    # Add a fully connected layer to output a vector of length 1296
    fc1 = Dense(1024, activation='relu')(x)
    fc1 = Dropout(0.2)(fc1)  # Add dropout
    fc2 = Dense(512, activation='relu')(fc1)
    fc2 = Dropout(0.2)(fc2)  # Add dropout
    output = Dense(1296, activation='softmax')(fc2)
    
    # Reshape output to a 36x36 one-hot encoded matrix
    reshaped_output = Reshape((36, 36))(output)
    sinkhorn_output = SinkhornLayer()(reshaped_output)
    
    # Define the Siamese model with 36 inputs and one output
    model = Model(inputs=inputs, outputs= sinkhorn_output)
    

    return model

# Create the Siamese network with input shape (50, 50, 3) and outputting a 36x36 one-hot encoded matrix
siamese_network = create_siamese_network()
siamese_network.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.00001),metrics=["accuracy"])


In [ ]:
siamese_network.fit(newx_train,newy_train,validation_data=(newx_test,newy_test),epochs=500)

In [ ]:
siamese_network.save_weights("model_landmarks.h5")